# Notebook Lecture 1: Introduction to System Modeling
© 2024 ETH Zurich, Mark Benazet Castells, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook is designed to introduce fundamental concepts in control systems engineering. It covers the basics of system modeling and introduces a simple controller. The notebook contains interactive examples to build an intuition of the subjects.

Authors:
- Jorit Geurts; jgeurts@ethz.ch
- Shubham Gupta; shugupta@ethz.ch

## Learning Objectives

In lecture 1 we saw that this course addresses the control of dynamical systems. As such, two of the course objectives are to understand how analyze and synthesize systems to achieve some goal (consider the reference examples from Lecture 1). However, to do so, we first need to understand how such systems can be represented mathematically (i.e., modeled).
Thus, we first introduce what is meant by a dynamical system, and what do we mean by modeling it, and then discuss modelling techniques for different types of systems.  

After completing this notebook, you should be able to:

1. Explain the concept of system modeling to another student.
2. Understand dynamic control systems in terms of inputs and outputs.
3. Understand the concept of "state" to model a system.
4. Derive a model of a dynamical system by hand and describe what their states, inputs and outputs are.
5. Understand how the output single of a dynamical system is computed, given an input signal and initial condition. (more formally introduced in Lecture 4) -- **to leave out?**
6. Understand the benefits and dangers of using feedback to control a dynamical system.

### Import the packages:

The following cell imports the required packages. Run it before running the rest of the notebook.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display

# Definitions


## Dynamical System

In general:<br>
- A dynamical (control) system generates one or many output signals given one or many input signals.

More formally:<br>
A dynamical control system is an operator, $\sum$, that takes an input signal, $u$, and produces an output signal $y$.
These dynamical systems have internal states, $x$, also referred to as *memory*, that describe the relationship between the input and output signal.<br>
In general, dynamic systems are arbitrated to any number of input and output signals. However, in this course we will consider Single Input Single Output (SISO) systems -- like in the above image.

<div style="text-align: center;">
    <img src="img/Dynamical_System.png" width="400">
</div>

To concretize the above, consider the two examples below:<br> **NOTE: look at examples below and see whether this is consistent.**
A pendulum swinging freely around a point is a dynamical system. In this case the input would be 0 and the output would be the angle of the pendulum.<br>
A pendulum swinging around a point that is attached to a motor is a dynamical control systems since we can control the input signal to the system. In this case the input would be the motor torque and the output will be the angle of the pendulum.

**Image here**

## Modeling

(System) Modeling for control systems is the act of formally representing the **relationship between the input(s) and output(s)** of a system.<br>
Usually this means that we want to write down the **Differential Equations (PDE/ODE)** that govern the underlying system we want to control.

Having a model of the system allows us to analyze the system, synthesis and implement in practical applications a control structure.

As mentioned in the lecture about modeling, *"All Models are wrong, but some are useful"*. This is due to the fact that in general it is not possible to exactly model all parts of the exactly (consider the heating system in your house, losses are present through the window, the window sills, the walls, etc...).
Thus, the underlying model will always be an approximation of reality and one needs to keep this in mind while creating and utilizing a model.

# Modeling Approaches

Let us now got through the steps of modeling a system.

### General Steps

At a high-level, the general steps to model a dynamical system are as follows:
1. Identify the relevant input and output variables. 
2. Derive the differential equation/s governing the dynamical system. 
3. Write the differential equations into state-space form. 

We will now introduce each of the above steps in more detail. 

#### 1. Identifying System Boundaries, Inputs and Outputs

First we define some relevant terminology:
1. **Inputs:** The inputs to a system define how the outside world affects the system itself. This includes both, the general environment and possibly some user with some goal of controlling the system for a specific task.   
For example while controlling the temperature of water in a faucet, it is affected by both the user controlling the knob and the heat dissipation in the plumbing.   
   
   Inputs can be separated into:
   * **Endogenous inputs**: Inputs which can be controlled. These are in general the control inputs (for example the faucet knob or the gas throttle from a car).
   * **Exogenous inputs**: Inputs which the user has no control over. These are usually (not always) disturbances (for example wind disturbances when flying a helicopter).

2. **Outputs:** The outputs of a system are the measurements or observations we make of the system.  

   These can be separated in:
   * **Measured outputs**, quantities we can measure directly (e.g., the speed of a car).
   * **Performance outputs**, which we can't measure directly but would also like to control (e.g., the average fuel efficiency of a car).

3. **System State/memory:**
The **memory** of a system, summarizes the effects of all past inputs. We will refer to this as the (internal) **state** of the system. It is a set of numbers and/or quantities describing the system. 
   
   For example, one could describe the state of a car by its position and velocity, $x(t) = [p(t), \dot p(t)]^T$. 


Some of the above aspects can be controlled while deriving a model, meaning we can choose what quantities will be captured in the model. Thus, to some extent we can choose the inputs to our system and the corresponding outputs to better suit our goal.

As an example, consider the system below: (Problem Set 1 Question 2 - **add image directly into here**)

In this model we included the drag force, roll resistance, traction (engine model) and gravity as inputs to the system.   
Another valid choice for inputs would be to ignore the drag force and only look at the roll resistance, traction and gravity.   
Whether this is a good choice or not depends on the overarching goal, but it would be a valid model of the system.

On the other hand one could make the model more complex and introduce the model that calculates the traction torque explicitly.
This would increase the scope of the system as new quantities such as temperatures and pressures inside the motor become relevant.
This again would be valid but maybe it would be to complicated for the task at hand.

Lastly we can also change the outputs of the system. We are not limited to measuring the velocity of the car, but we could instead measure acceleration, or include a GPS device and measure the position explicitly. Again, these are objective dependent considerations that vary according to the context of the problem.

Since the principal objective of the course is to teach how to control said systems, it will be clear by the context what the relevant inputs, outputs and system states are. 

#### 2. Writing Down the Differential Equations

Once the system boundaries, inputs and outputs are defined the differential equations of the system need to be derived.   
There are different approaches to doing this, but often this is done using so called storage variables (such as energy, temperature, mass, momentum, cash, etc), and how different inflows and outflows change these storage variables (mass flow, power heat flow, payments, etc.). The storage variables are usually system states and the inflows and outflows are usually the inputs to the system. Once these storage variables are defined and the inflows and outflows are known we can setup the differential equation using:
$$\frac{d}{dt}\text{storage} = \sum \text{inflows} - \sum \text{outflows}$$
Most physical laws can be interpreted or rewritten as a variation of the storage approach. Thus it is often more convenient to use these laws to directly setup the differential equations.

Following are some of the common formulas used (with $\dot \square$  indicating the time derivative):

**Newton's 2nd Law for Linear Translation:**
$$
\sum F(t) = m a(t) = m \dot v(t) = m \ddot x(t)
$$
with $\sum F(t)$ being the sum of all forces, $m$ being the mass of the object, $a(t)$ the acceleration, $v(t)$ the velocity and $x(t)$ the position.

**Newton's 2nd Law for Rotation:**
$$
\sum T(t) = J \ddot \theta(t) = J \dot \omega (t)
$$
with $\sum T(t)$ being the sum of all torques, $J$ being the Moment of Inertia, $\omega(t)$ being the rotational velocity, and $\theta(t)$ being the rotation angle.

**Heat Transfer (Fourier's Law):**

A heat flow from surface i too surface j of a wall is proportional to the difference of temperatures: 
$$
\dot Q_{ij}(t) = c_{ij} (T_i(t)-T_j(t))
$$
with $c_{ij}$ being a constant related to the two surfaces and the wall between.<br>
Having a heat flow into a room from different surfaces we have the following temperature equation:
$$
C \dot T(t) = \sum \dot Q(t)
$$
with $C$ being the thermal capacity of that room.

**Inductance:** *Not exam relevant*
$$
U(t) = L \dot I(t)
$$
with $U(t)$ being the voltage, $L$ the inductance and $I(t)$ the current.

**Capacitance:** *Not exam relevant*
$$
I(t) = C \dot U(t)
$$
with $U(t)$ being the voltage, $C$ the capacitance and $I(t)$ the current.

#### 3. Writing Down in Standard Form

Every control system has different quantities to keep track of, different inputs, outputs, state and most importantly different orders of derivatives.
As this variety in models(variable names, derivative orders, inputs, output) would make it difficult to come up with unified theories that can be applied to all control systems it is important to write down the differential equations in a standard form, which looks like this:
$$
\dot x (t) = f(x(t), u(t))\\
y(t) = h(x(t), u(t))
$$
Important to not here that $x(t)$, $u(t)$ and $y(t)$ are vectors and on the left hand side we only have the first derivative, i.e.:
$$
\begin{bmatrix}
    \dot x_1(t)
    \\
    \vdots
    \\
    \dot x_N(t)
\end{bmatrix}
=
\begin{bmatrix}
    f_1(x(t), u(t))
    \\
    \vdots
    \\
    f_N(x(t), u(t))
\end{bmatrix}
$$
$$
\begin{bmatrix}
    y_1(t)
    \\
    \vdots
    \\
    y_N(t)
\end{bmatrix}
=
\begin{bmatrix}
    h_1(x(t), u(t))
    \\
    \vdots
    \\
    h_N(x(t), u(t))
\end{bmatrix}\\
$$
This ensures that all system have the same form and results are easily transferable.
In this formulation $f(x(t), u(t))$ is called the dynamics model and $h(x(t), u(t))$ is called the measurement model.

Usually the systems are not standard form and mostly include higher order derivatives. To convert these higher order systems to standard form we need to use a trick from Linear Algebra that every higher derivative can be reformulated as a system of first order derivatives.

Lets take this example differential equation here
 $$\frac{d^2}{dt^2} \theta + \frac{g}{l}sin(\theta) = 0$$  
 $$ y = \theta $$

At a first glance it is not in the standard form, but looks similar. The main problem is the derivative. So how do we resolve that?   

*We introduce an extra state*. 

So now we define our state to be $$ x = [\theta, \dot \theta]^T$$ and we can write it as 
$$ \dot x_1 = \dot \theta = x_2 $$
$$ \dot x_2 = \ddot \theta =  -\frac{g}{l} \sin(x_1)$$
$$ y = x_1 $$

The system is now in standard form. It is important to note that a system definition is no unique, one can add intermediate states. This overprescription of a system will be introduced later in the lectures.

## Example System
Let us now go through these three steps in a simple spring damper system. <br>
*Note: These kind of spring damper systems are important for measurement systems, eigenfrequency calculation of mechanical systems, modeling material properties or calculating the damping properties of systems (for example an earthquake damper in a building).*

<img src="img/Spring_Damper_System.png" width="800">

Our goal is to model the equation which describe the movement of the point mass. With this model we then want to create a controller which allows us to follow a reference position.
This can be achieved by the force $F_c(t)$ that we can control.

### System A:
In the system A we have a spring where its natural state is at $x=0$ and spring constant $k$. The mass hanging on the spring is $m$ and there acts a force $F_c$ on the point mass. This force is time variant and can be controlled by us. For the measurement we have included a force sensor on the base of the spring.

#### Step 1: Defining the boundaries, inputs and outputs
For us only the position of the mass is relevant, thus this is the only quantity to keep track of. The easiest way to model this system is to create a free body diagram of the point mass and interpret the forces (Spring Law: $F=k\Delta x$, where k is a constant and $\Delta x$ is the deviation from the natural state) as inputs to the system. For the output we only have the force measurement at the base of the spring.

<img src="img/Free_Body_Diagram_2.png" width="400">

#### Step 2: Setup the differential equations
If we now use Newtons 2nd Law for linear motion as mentioned above we get the following equation:
$$
\sum F = m \ddot x \Rightarrow F_c(t) + mg - k x(t) = m \ddot x(t)
$$
We can rewrite this and get:
$$
\ddot x = \frac{1}{m} (F_c(t) + mg - kx(t))
$$
For the output we have a force measurement that is proportional to x(t)
$$
y(t) = k \cdot x(t)
$$ 

#### Step 3: Write down in standard form
Since we have a second derivative our system is not yet in standard form. We first define our input as
$$
u(t) = F_c(t)
$$
Next we define the following states (we replace $x(t)$ in the definition of the standard form with $z(t)$ to avoid confusion):
$$
z_1(t) = x(t), \quad z_2(t) = \dot x(t) 
\Rightarrow z(t) = 
\begin{bmatrix}
    z_1(t) &
    z_2(t)
\end{bmatrix}^T
$$
With this we get the following:
$$
\dot z(t) = 
\begin{bmatrix}
    \dot z_1(t) \\
    \dot z_2(t)
\end{bmatrix}
\begin{bmatrix}
    \dot x(t) \\
    \ddot x(t)
\end{bmatrix}
= 
\begin{bmatrix}
    \dot z_2(t) \\
    \frac{1}{m} (u(t) + mg - kz_1(t))
\end{bmatrix}
$$
And the output is:
$$
y(t) = k \cdot z_1(t)
$$
Thus our system is described by:
$$
\dot z(t) = 
\begin{bmatrix}
    \dot z_2(t) \\
    \frac{1}{m} (u(t) + mg - kz_1(t))
\end{bmatrix}
$$
$$
y(t) = k \cdot z_1(t)
$$

As an exercise you can now fill the blanks for system B.

### System B: Exercise
In the system B we have a spring where its natural state is at $x=0$ and spring constant $k$. The mass hanging on the spring is $m$ and there acts a force $F_c$ on the point mass. This force is time variant and can be controlled by us. Additionally we have a damper in the system, with damper coefficient $b$ (Damper Law: $F=b \dot x$, where $b$ is a constant and $\dot x$ is the velocity of the spring). This time we can directly measure the position of $x$

#### Step 1: Defining the boundaries, inputs and outputs
Create the free body diagram and draw the forces accordingly.

#### Step 2: Setup the differential equations
Using Newtons 2nd Law for linear motion as mentioned above we get:
$$
\sum F = m \ddot x \\
\Rightarrow \square = \square
$$
We can rewrite this and get:
$$
\ddot x(t) = \square
$$
For the output we have a force measurement that is proportional to x(t)
$$
y(t) = \square
$$ 

#### Step 3: Write down in standard form
Since we have a second derivative our system is not yet in standard form. We first define our input as
$$
u(t) = \square
$$
Next we define the following states (we replace $x(t)$ in the definition of the standard from with $z(t)$ to avoid confusion):
$$
z(t) = 
\square
$$
With this we get the following:
$$
\dot z(t) = 
\square
$$
And the output is:
$$
y(t) = \square
$$


### System B: Solution
In the system B we have a spring where its natural state is at $x=0$ and spring constant $k$. The mass hanging on the spring is $m$ and there acts a force $F_c$ on the point mass. This force is time variant and can be controlled by us. Additionally we have a damper in the system, with damper coefficient $b$. This time we can directly measure the position of $x$

#### Step 1: Defining the boundaries, inputs and outputs
Again only the position of the mass is relevant, thus this is the only quantity to keep track of. The easiest way to model this system is to create a free body diagram of the point mass and interpret the forces (Damper Law: $F=b \dot x$, where $b$ is a constant and $\dot x$ is the velocity of the spring) as inputs to the system. For the output we have the position of the point mass.
To end up in this formulation one first defines the inputs as a input vector $u(t)$, then define the outputs/measurements as an output vector $y(t)$ and lastly define the system states as state vector $x(t)$.   

<img src="img/Free_Body_Diagramm.png" width="800">

#### Step 2: Setup the differential equations
Using Newtons 2nd Law for linear motion as mentioned above we get:
$$
\sum F = m \ddot x \\
\Rightarrow F_c(t) + mg - kx(t) - b\dot x(t) = m \ddot x(t)
$$
We can rewrite this and get:
$$
\ddot x(t) = \frac{1}{m}(F_x(t)+mg-kx(t)-b\dot x(t))
$$
For the output we have a force measurement that is proportional to x(t)
$$
y(t) = x(t)
$$ 

And we have them is the standard form (The first two equation for the dynamics model and the last euation is the measuement model).  
This is one way. There is other ways as well to put the derived differential equations in the stsandard form.
#### Step 3: Write down in standard form
Since we have a second derivative our system is not yet in standard form. We first define our input as
$$
u(t) = F_c(t)
$$
Next we define the following states (we replace $x(t)$ in the definition of the standard from with $z(t)$ to avoid confusion):
$$
z(t) = 
\begin{bmatrix}
    z_1(t) \\
    z_2(t)
\end{bmatrix}
= \begin{bmatrix}
    x(t) \\
    \dot x(t)
\end{bmatrix}
$$
With this we get the following:
$$
\dot z(t) = 
\begin{bmatrix}
    z_2(t) \\
    \frac{1}{m}(u(t)+mg-kz_1(t)-b\dot z_2(t))
\end{bmatrix}
$$
And the output is:
$$
y(t) = z_1(t)
$$


## Computing the Output Signal
Once we have a system in standard form we often would like to compute the output of that system for different inputs.
However calculating the output in a closed form solution ($x(t) = g(x_0, u(t))$) is often very difficult or sometimes impossible. <br>
However we know that over some time $\Delta t$ the system evolves as follows:
$$
x(t+\Delta t) = \int_{t}^{\Delta t} f(x(\tau), u(\tau)) d\tau
$$
This calculation is also not trivial but for sufficiently small $\Delta t$ we can approximate this integral using:
$$
x(t+\Delta t) \approx x(t) + \Delta t \cdot f(x(t), u(t))
$$
*Note: An alternative interpretation of this formula comes from the approximation of the derivative (rearranging this formula results in the same as above):*
$$
\dot x = f(x(t), u(t)) \approx \frac{x(t+\Delta t)- x(t)}{\Delta t}
$$


Having this approximation we can write simulators to calculate the output of the system given some input signal.
For a simple simulator you can check out the following code that calculates the trajectory of the spring damper system.

In [2]:
class simulator:
    
    def __init__(self):
        # define the parameters of your system
        self.m = 1                  #kg
        self.g = 9.81               #m/s^2
        self.t_end = 15             #s

    def nonlinear_dynamics(self, x, u, t, damper_on, k, b):
        # define the nonlinear dynamics f(x(t), u(t)) of the system
        return np.array([x[1], self.g - k/self.m*x[0] + u/self.m - b/self.m*x[1]*damper_on])
    
    def euler(self, x_0, k, b, damper_on, delta_t):
        # simple euler integration simulator
        x = np.copy(x_0)
        x_history = np.copy(x_0)
        t = 0
        N = int(self.t_end / delta_t)
        c = 0
        while c < N:
            u = 0
            x = x + delta_t * self.nonlinear_dynamics(x, u, t, damper_on, k, b)
            t += delta_t
            c += 1
            x_history = np.vstack((x_history, x))
        return x_history
    
model = simulator()
def run_sim(k, b, damper_on=0, dt=1):
    x_0 = np.array([0, 0])
    x_history = model.euler(x_0, k, b, damper_on, dt/1000)
    return x_history
    

### Interactive Exercise
In the example you can vary the spring constant and damping constant and see how these affect the system.
Additionally a slider which indicates the step size $\Delta t$. Playing around with this shows that one has to be careful when relying on simulations as strange things can be output.
When having a to small time step the simulation take to long to calculate and even crash the simulation. When the step size is to large the system shows an unstable output due to discretization errors.
These phenomena are not important for this course but this should be a reminder for working with simulations.

In [3]:
## Code for interactive plot
# Initialize a display object
output = widgets.Output()
# Function to update the plot and title
def update_plot(k_slider, b_slider, dt_slider):
    with output:
        output.clear_output(wait=True)  # Clear the previous output
        # two subplots next to each other with given figure size:
        fig, axs = plt.subplots(1, 2, figsize=(20,5))
        x_out = run_sim(k=k_slider, b=b_slider, damper_on=0, dt=dt_slider)
        axs[0].plot(x_out[:,0], label='No Damper', color='red')
        axs[1].plot(x_out[:,1], label='No Damper', color='red')
        x_out = run_sim(k=k_slider, b=b_slider, damper_on=1, dt=dt_slider)
        axs[0].plot(x_out[:,0], label='Damper', color='orange')
        axs[1].plot(x_out[:,1], label='Damper', color='orange')

        # x lable with latex format of theta
        axs[0].set_xlabel(r'Time: t[s]')
        axs[0].set_ylabel(r'Position: $x(t) [m]$')
        axs[1].set_xlabel(r'Time: t[s]')
        axs[1].set_ylabel(r'Velocity: $\dot x(t)[m/s]$')
        axs[0].set_title('Time response (dt = '+str(dt_slider/1000)+'[s]'+")")
        
        for ax in axs:
            xticks = ax.get_xticks()  # Get current x-ticks
            ax.set_xticks(xticks)  # Fix the ticks first
            ax.set_xticklabels([f'{int(tick*dt_slider/1000)}' for tick in xticks])  # Divide x-ticks by 100

        axs[0].legend()
        axs[1].legend()
        axs[0].grid()
        axs[1].grid()
        plt.show()

# Create a slider for Kp and Kd
k_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description="Spring constant")
b_slider = widgets.FloatSlider(value=1, min=0, max=5, step=0.1, description="Damping coefficient")
dt_slider = widgets.FloatSlider(value=10, min=1, max=500, step=0.1, description="Time step [ms]")

# Arrange the checkboxes box and slider in a horizontal box
controls_box = widgets.VBox([k_slider, b_slider, dt_slider])

# Link the widgets to the update_plot function
widgets.interactive_output(update_plot, {
    'k_slider': k_slider,
    'b_slider': b_slider,
    'dt_slider': dt_slider
})

# Display the controls and the plot
display(controls_box, output)

Output()

## PS1 Exercise 3

Following is a simulation of the reverse pendulum exercise (PS1 Exercise 3). 
When you first run the code you can see how the system naturally behaves. It starts upright until at $t=3$ an impulse torque notches the pendulum out of its unstable equilibrium. The pendulum then swings down and approaches its stable equilibrium (being at 180° ($\pi$ rad)). This is what we would expect from this system.

Now the goal is to introduce a feedback controller to stabilize the system to an upright position. 
For this we will first introduce the simplest controller imaginable. 
Lets say we want the system to be at the desired angle $\theta_{ref} = 0$.
We can then define the error of the system as the deviation of the angle from the reference angle:
$$
e(t) = \theta_{ref}-\theta(t)
$$
We can then say the controller should input a control signal proportional to the error:
$$
u_p(t) = K_p \cdot e(t)
$$.
To see what happens you can turn on the controller and increase the $K_p$ slider.
One can see that in the beginning the control is not able to reach the desired state of $\theta = 0$ since to little effort is made.
As one increases the $K_p$ gain the controller approaches the desired state of $\theta = 0$. This is certainly a controller that works but one can see that the controller overshoots. The larger the gain the higher the overshoot. This overshooting (going above or below the desired goal) can be mitigated by introducing a damping term in the controller. For this we calculate the rate of change of the error (i.e. are we make a lot of progress towards the right direction or not). We define:
$$
\dot e(t) = \frac{d}{dt}(\theta_{ref}- \theta(t)) \underbrace{=}_{\text{since } \dot \theta_{ref}\text{ is zero}} - \dot \theta(t)
$$
We now introduce this term into the control input as follows:
$$
u_{PD}(t) = K_p \cdot e(t) + K_d \cdot \dot e(t)
$$
This is a so called PD-Controller which you will encounter later in this course. 
To see the effect increase the $K_d$ gain slider and see how the system overshoot is smaller.
One can interpret this as a sort of damping, when we are moving fast towards the desired goal the controller backs of to "gently" approach the desired goal.
Feel free to play around with different values of $K_p$ and $K_d$.

And lastly one can also see the effect of feedback being negative. If the $K_d$ gain is zero and the $K_p$ gain is to large the system starts to become unstable.
This nicely demonstrates that one has to be careful when implementing feedback as in the worst case a rogue controller will damage or destroy your system.

In [4]:
class simulator:
    
    def __init__(self):
        # define the parameters of your system
        self.m = 1                  #kg
        self.g = 9.81               #m/s^2
        self.l = 1                  #m
        self.cf = 0.5               #Nms/rad
        self.impulse_torque = 10    #Nm
        self.impulse_duration = 0.1 #s
        self.impulse_time = 3       #s
        self.t_end = 20             #s
        self.delta_t = 0.01         #s

    def nonlinear_dynamics(self, x, u, t):
        # define the nonlinear dynamics f(x(t), u(t)) of the system
        disturbance = 0
        if t > self.impulse_time and t < self.impulse_time + self.impulse_duration:
            disturbance = self.impulse_torque
        return np.array([x[1],
                        3*self.g/(2*self.l)*np.sin(x[0]) 
                        - 3*self.cf/(self.m*self.l**2)*x[1] 
                        + 3/(self.m*self.l**2)*u
                        + 3/(self.m*self.l**2)*disturbance])
    
    def euler(self, control_on, x_0, Kp, Kd):
        # simple euler integration simulator
        x = np.copy(x_0)
        x_history = np.copy(x_0)
        t = 0
        if control_on:
            while t < self.t_end:
                u = self.control(x, t, Kp, Kd)
                x = x + self.delta_t * self.nonlinear_dynamics(x, u, t)
                t += self.delta_t
                x_history = np.vstack((x_history, x))
            return x_history
        else:
            while t < self.t_end:
                x = x + self.delta_t * self.nonlinear_dynamics(x, 0, t)
                t += self.delta_t
                x_history = np.vstack((x_history, x))
            return x_history
    
    def control(self, x, t, Kp, Kd):
        # simple PD controller
        u = - Kp*x[0] - Kd*x[1]
        return u
    
model = simulator()
def run_sim(control_on, K_p, K_d):
    x_0 = np.array([0, 0])
    x_history = model.euler(control_on, x_0, K_p, K_d)
    return x_history
    

In [5]:
## Code for interactive plot
# Initialize a display object
output = widgets.Output()

# Function to update the plot and title
def update_plot(control_off_checkbox, control_on_checkbox, Kp_slider, Kd_slider):
    with output:
        output.clear_output(wait=True)  # Clear the previous output
        # two subplots next to each other with given figure size:
        fig, axs = plt.subplots(1, 2, figsize=(20,5))
        
        if control_off_checkbox:
            x_out = run_sim(control_on=False, K_p=Kp_slider, K_d=Kd_slider)
            axs[0].plot(x_out[:,0], label='No Feedback', color='red')
            axs[1].plot(x_out[:,1], label='No Feedback', color='red')
        
        if control_on_checkbox:
            x_out = run_sim(control_on=True, K_p=Kp_slider, K_d=Kd_slider)
            axs[0].plot(x_out[:,0], label='With Feedback', color='blue')
            axs[1].plot(x_out[:,1], label='With Feedback', color='blue')
        
        # x lable with latex format of theta
        axs[0].set_xlabel(r'Time: t[s]')
        axs[0].set_ylabel(r'Angle: $\theta(t) [rad]$')
        axs[1].set_xlabel(r'Time: t[s]')
        axs[1].set_ylabel(r'Angular Velocity: $\dot\theta(t)[rad/s]$')
        for ax in axs:
            xticks = ax.get_xticks()  # Get current x-ticks
            ax.set_xticks(xticks)  # Fix the ticks first
            ax.set_xticklabels([f'{int(tick/100)}' for tick in xticks])  # Divide x-ticks by 100

        if control_off_checkbox or control_on_checkbox: # avoids legend error if none of the functions are selected
            axs[0].legend()
            axs[1].legend()
        axs[0].grid()
        axs[1].grid()
        plt.show()

# Create checkboxes to show controlled and uncontrolled plots
control_off_checkbox = widgets.Checkbox(value=True, description='Control off')
control_on_checkbox = widgets.Checkbox(value=False, description='Control on')

# Create a slider for Kp and Kd
Kp_slider = widgets.FloatSlider(value=0, min=-1, max=70, step=0.1, description="K_p gain")
Kd_slider = widgets.FloatSlider(value=0, min=-1, max=2, step=0.1, description="K_d gain")

# Arrange checkboxes in a vertical box
checkboxes_box = widgets.VBox([control_off_checkbox, control_on_checkbox])

# Arrange the checkboxes box and slider in a horizontal box
controls_box = widgets.HBox([checkboxes_box, widgets.VBox([Kp_slider, Kd_slider])])

# Link the widgets to the update_plot function
widgets.interactive_output(update_plot, {
    'control_off_checkbox': control_off_checkbox,
    'control_on_checkbox': control_on_checkbox,
    'Kp_slider': Kp_slider,
    'Kd_slider': Kd_slider
})

# Display the controls and the plot
display(controls_box, output)

Output()